In [1]:
# 라이브러리 이미지 임포트
import tensorflow as tf

In [2]:
# tf.constant로 상수를 Hello 변수에 저장
hello = tf.constant('Hello, TensorFlow!')
print(hello)

# hello가 텐서플로의 Tensor 라는 자료형이고 상수를 담고 있음. 랭크와 셰이프 개념.   셰이프는 [1,2,3] 이 한 묶음. 랭크는 이를 밖에서 묶은 개념으로 1


Tensor("Const:0", shape=(), dtype=string)


In [3]:
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a, b)
print(c)

# 그래프는 텐서들의 연산 모음으로 c=tf.add는 지연실행(lazy evaluation) 된 상황.
# 아래 sess.close() 에서 실제 실행 됨.

Tensor("Add:0", shape=(), dtype=int32)


In [4]:
sess = tf.Session()

print(sess.run(hello))
print(sess.run([a, b, c]))

sess.close()

b'Hello, TensorFlow!'
[10, 32, 42]


# 기초2. 플레이스홀더와 변수

In [6]:
# 그래프에 사용할 입력값을 나중에 받기 위해 사용하는 매개변수(Parameter)이다. 
# 변수는 그래프를 최적화하는 용도로 텐서플로가 학습한 결과를 갱신하기 위해 사용하는 변수

X = tf.placeholder(tf.float32, [None, 3])
print(X)

Tensor("Placeholder_1:0", shape=(?, 3), dtype=float32)


In [7]:
x_data = [[1,2,3],[4,5,6]]   # 랭크가 2, 셰이프는 2,3 (중괄호 안에 들어있는 인자 개수)

In [9]:
W = tf.Variable(tf.random_normal([3,2]))
b = tf.Variable(tf.random_normal([2,1]))

In [10]:
W = tf.Variable([[0.1, 0.1], [0.2, 0.2], [0.3, 0.3]])

In [11]:
expr = tf.matmul(X, W) + b

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())   # 앞에서 정의한 변수들을 초기화하는 함수. 기존에 학습한 값들을 가져와서 실행하는 게 아니라면 연산 실행 전에 반드시 이 함수를 이용해 변수들을 초기화 해야 한다.

print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))
print("=== expr ===")
print(sess.run(expr, feed_dict={X: x_data}))

sess.close()

=== x_data ===
[[1, 2, 3], [4, 5, 6]]
=== W ===
[[0.1 0.1]
 [0.2 0.2]
 [0.3 0.3]]
=== b ===
[[-0.5961132 ]
 [-0.16502537]]
=== expr ===
[[0.8038869 0.8038869]
 [3.0349746 3.0349746]]


### 연습. 선형 회귀 모델 구현하기
 - 선형 회귀란 간단하게 말해, 주어진 x와 y의 값을 가지고 서로간의 관계를 파악하는 것
 - 이 관계를 알고 나면 새로운 x 값이 주어졌을 때 y 값을 쉽게 알수 있다.

In [25]:
x_data = [1, 2, 3]
y_data = [1, 2, 3]

In [26]:
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

In [27]:
X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")

In [28]:
hypothesis = W * X + b

In [29]:
# 손실함수(loss function) : 한 쌍(x, y)의 데이터에 대한 손실값을 계산하는 함수
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [39]:
# Gradient Descent(경사하강법) 최적화 Method를 이용해 손실값을 최소화 하는 연산그래프
#  -> 최적화 함수란 가중치와 편향 값을 변경해가면서 손실값을 최소화하는 가장 최적화된 가중치와 편향값을 찾아주는 함수.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1) # 학습률과 같이 학습을 진행하는 과정에서 영향을 주는 변수를 '하이퍼파라미터' 라 한다.
train_op = optimizer.minimize(cost)

# 즉, 함수의 기울기를 구하고 기울기가 낮은 쪽으로 계속 이동시키면서 최적화의 값을 찾아나가는 방법

In [43]:
# 세션을 생성하고 초기화합니다.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # 최적화를 100번 수행합니다.
    for step in range(100):
        # sess.run 을 통해 train_op 와 cost 그래프를 계산합니다.
        # 이 때, 가설 수식에 넣어야 할 실제값을 feed_dict 을 통해 전달합니다.
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y: y_data})

        print(step, cost_val, sess.run(W), sess.run(b))


    # 최적화가 완료된 모델에 테스트 값을 넣고 결과가 잘 나오는지 확인해봅니다.
    # X가 5, 2.5 일 때 Y는 몇인가를 예측해서 출력.
    print("\n===Test===")
    print("X: 5, Y:", sess.run(hypothesis, feed_dict={X: 5}))
    print("X: 2.5, Y:", sess.run(hypothesis, feed_dict={X: 2.5}))

0 7.597962 [0.8770337] [0.5962786]
1 0.13282286 [0.7532908] [0.5262094]
2 0.041652214 [0.77306896] [0.51965123]
3 0.038660012 [0.77701074] [0.5064934]
4 0.036811527 [0.7825367] [0.4943904]
5 0.0350628 [0.7877463] [0.48249763]
6 0.033397287 [0.79285073] [0.47089958]
7 0.03181089 [0.7978302] [0.45957938]
8 0.030299827 [0.8026902] [0.4485314]
9 0.028860597 [0.8074335] [0.43774903]
10 0.027489712 [0.8120627] [0.42722583]
11 0.026183913 [0.81658053] [0.4169556]
12 0.024940133 [0.8209898] [0.40693226]
13 0.023755474 [0.82529306] [0.3971499]
14 0.02262708 [0.8294929] [0.3876027]
15 0.021552265 [0.8335918] [0.378285]
16 0.020528538 [0.8375922] [0.3691913]
17 0.019553393 [0.8414962] [0.36031613]
18 0.018624606 [0.84530663] [0.3516544]
19 0.017739927 [0.84902537] [0.34320086]
20 0.016897233 [0.85265464] [0.33495054]
21 0.01609463 [0.85619676] [0.32689857]
22 0.015330125 [0.8596537] [0.31904015]
23 0.014601931 [0.8630276] [0.31137064]
24 0.013908318 [0.86632025] [0.3038855]
25 0.013247655 [0.8695